In [1]:
import os, sys
sys.path.append("/home/alexanderalbizu")
sys.path.append("/home/alexanderalbizu/.local/bin")
os.environ["WANDB_NOTEBOOK_NAME"] = "CNN.ipynb"
#!python setup.py develop 
# !pip install wandb
# !pip install 'monai[all]'
#!pip -q install vit_pytorch
#!pip -q install linformer

In [2]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import glob, math, os, shutil, tempfile, time, monai, torch, random

import wandb as wb
from enum import Enum
import pandas as pd
import numpy as np
import nibabel as nib
from torch import nn
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from monai.networks.utils import eval_mode
from monai.config import print_config
from monai.data import (
    CacheDataset,
    DataLoader,
    ThreadDataLoader,
    ImageDataset,
    Dataset,
    decollate_batch,
)
from sklearn.metrics import (
    classification_report, confusion_matrix,
    ConfusionMatrixDisplay
)
from sklearn.model_selection import train_test_split
from monai.inferers import sliding_window_inference
from monai.losses import DiceLoss, DiceCELoss
from monai.metrics import DiceMetric
from monai.networks.layers import Act, Norm
from monai.transforms import (
    AddChannel,
    AsChannelFirst,
    Compose,
    RandGaussianNoise,
    Resize,
    RemoveRepeatedChannel,
    Orientation,
    RandRotate90,
    RandBiasField,
    ScaleIntensity,
    ToDevice,
    EnsureType,
)
from monai.utils import set_determinism

pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print_config()
wb.login(); # 7e5f63e5846f29b034d98806712ab047df76834d

MONAI version: 0.8.1+331.g6a301d51.dirty
Numpy version: 1.20.1
Pytorch version: 1.9.0+cu111
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 6a301d51fbbb1803b7349a85c9bfa398f19ee0f9
MONAI __file__: /home/alexanderalbizu/MONAI/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.8
Nibabel version: 3.2.1
scikit-image version: 0.19.1
Pillow version: 9.1.1
Tensorboard version: 1.15.0+nv
gdown version: 4.4.0
TorchVision version: 0.10.0+cu111
tqdm version: 4.53.0
lmdb version: 1.2.1
psutil version: 5.8.0
pandas version: 1.1.4
einops version: 0.4.1
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: 1.26.1
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



wandb: Currently logged in as: aalbizu. Use `wandb login --relogin` to force relogin


In [3]:
# Set data directory
rootDir = '/blue/camctrp/working/Alejandro/StimBrain/'
dims = np.array([182,218,182])
batch_size=32
im_size = (128,128)
seed = 42
subs = np.array([9009, 9015, 9022, 9040, 9044, 9045, 9051, 9021, 9023, 9031, 9032, 9047, 9048, 9054]) # Subjects
beh = np.array([32, 26, 19, 20, 20, 17, 15, 2, 4, 6, 10, 16, 18, 7]) # Targeted Behavior Change
lr = 1e-5
wd = 0
K = 7 # MUST BE A FACTOR OF NUM SUBS
datatype = "Jxyz2D"
pat_size = (25,25)
pretrain = False;

# start a typical PyTorch training
val_interval = 1
epoch_loss_values = [] # Pre-Allocate
epoch_acc_values = [] # Pre-Allocate
max_epochs = 100
num_cores = int(os.environ["SLURM_CPUS_PER_TASK"]);

# Define Group Labels as above/below Median
l = np.int64(np.zeros([len(beh),1]));
l[beh >= np.median(beh)] = 1; # Responders
l[beh < np.median(beh)] = 0; # Non Responders


# INITIATE WEIGHTS AND BIASES
wb.init(project="SMART-CNN-2D_II",
        config={
            "batch_size": batch_size,
            "n_channels": 3,
            "n_epoch": max_epochs,
            "folds": K,
            "image_size": im_size,
            "pre-trained": pretrain,
              "patch_size": pat_size,
            "learning_rate": lr,
            "network": "ResNet10",
            "weight_decay": wd,
            "dataset": "SB",
        })
run_id = wb.run.name;

class Diagnosis(Enum):
    NonResponder = 0
    Responder = 1
    
def save_model(n_epoch, save_path, run_id):
    lastmodel = f"{save_path}-{run_id}-fold{f}.pth"
    torch.save(
        {
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "best_valid_score": best_valid_score,
            "n_epoch": n_epoch,
        },
        lastmodel,
    )

# Define Train/Validation/Test Set # WIP PERFORM 5 FOLD CROSS VALIDATION

# nontest_subs, test_subs, y_nontest, y_test = train_test_split(subs, l, test_size=0.10, stratify=l, random_state=seed);
butt = [0,1];
testIdx = np.array(random.sample(list(np.arange(subs.shape[0])),subs.shape[0]));
alltargs = []; allpreds = []; allscores = []; # Pre-Allocate
for f in range(K):
    print("Beginning Fold ", f+1)
    test_subs = subs[testIdx[butt]]; print('test subs: ', test_subs)
    nontest_subs = subs[np.logical_not(np.isin(subs,test_subs))]
    y_nontest = l[np.logical_not(np.isin(subs,test_subs))]
    train_subs, valid_subs, y_train, y_valid = train_test_split(nontest_subs, y_nontest, test_size=0.10, stratify=y_nontest, random_state=seed); print('valid subs: ',valid_subs) 

    test_list = []; valid_list = []; train_list = [];
    test_label = []; valid_label = []; train_label = [];
    for s in range(subs.shape[0]):
        for n in range(dims[2]):
                if n < 50 or n > 150: continue
                if np.isin(test_subs,subs[s]).any():
                    test_list.append(os.path.join(rootDir,'FS6.0_sub-'+str(subs[s])+'_ses01','sub-'+str(subs[s])+'_n'+str(n+1)+'_Jxyz.png'));
                    test_label.append(l[s]);
                elif np.isin(train_subs,subs[s]).any():
                    train_list.append(os.path.join(rootDir,'FS6.0_sub-'+str(subs[s])+'_ses01','sub-'+str(subs[s])+'_n'+str(n+1)+'_Jxyz.png'));
                    train_label.append(l[s]);
                elif np.isin(valid_subs,subs[s]).any():
                    valid_list.append(os.path.join(rootDir,'FS6.0_sub-'+str(subs[s])+'_ses01','sub-'+str(subs[s])+'_n'+str(n+1)+'_Jxyz.png'));
                    valid_label.append(l[s]);

    train_list = np.array(train_list); train_label = np.array(train_label);                  
    valid_list = np.array(valid_list); valid_label = np.array(valid_label);                  
    test_list = np.array(test_list); test_label = np.array(test_label);                  
                
    # Plot Labels
#     plt.rcParams['figure.figsize'] = [20, 20]; plt.imshow(np.asarray([train_list.T])); plt.axis('off'); plt.title('Class Labels');

# def seed_everything(seed):
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)
#     torch.backends.cudnn.deterministic = True
    
# seed_everything(seed)    

# Oversampling the Unbalnaced Class
# train_list = np.array(np.hstack([[train_list.T],[train_list.T]]))[0,:];
# train_label = np.array([np.int64(np.hstack([train_label[:,0],train_label[:,0]]))]).T;

    print('train case split: ',sum(train_label)[0],':',len(train_label)-sum(train_label)[0])
    print('valid case split: ',sum(valid_label)[0],':',len(valid_label)-sum(valid_label)[0])
    print('test case split: ',sum(test_label)[0],':',len(test_label)-sum(test_label)[0],'\n')

# Plot Responder Mean
# plt.rcParams['figure.figsize'] = [5,5];
# print(train_list[200])
# img = plt.imread(train_list[200]); print(img.shape) # Load Each Electrode
# plt.imshow(img, cmap="turbo", vmin=0, vmax=1e-100); plt.axis('off'); # Plot

# Represent labels in one-hot format for binary classifier training,
# BCEWithLogitsLoss requires target to have same shape as input
# labels = nn.functional.one_hot(torch.as_tensor(lab).T).long();
    train_lab = nn.functional.one_hot(torch.as_tensor(train_label).T).long(); 
    valid_lab = nn.functional.one_hot(torch.as_tensor(valid_label).T).long();
    test_lab = nn.functional.one_hot(torch.as_tensor(test_label).T).long();

    # Define transforms
    train_transforms = Compose([
        AsChannelFirst(channel_dim=2),
    #     AddChannel(),
        Resize(im_size),
    #     RandBiasField(),
        RandGaussianNoise(prob=0.1), 
    #     ScaleIntensity(minv=0.0, maxv=1.0),
        EnsureType(data_type='tensor')]);

    val_transforms = Compose([
        AsChannelFirst(channel_dim=2),
    #     AddChannel(),
        Resize(im_size),
    #     ScaleIntensity(minv=0.0, maxv=1.0),
        EnsureType(data_type='tensor')]);

    # Define nifti dataset, data loader
    check_ds = ImageDataset(image_files=train_list, labels=train_lab[0,:,:], transform=train_transforms);
    check_loader = DataLoader(check_ds, batch_size=1, num_workers=num_cores, pin_memory=pin_memory); 

    # im, label = monai.utils.misc.first(check_loader); print(im.shape, label, label.shape)
    # plt.imshow(im[0,:,:,:].T, cmap="turbo", origin="lower"); plt.axis('off'); del check_ds, check_loader, im, label;

    # create a training data loader
    train_ds = ImageDataset(image_files=train_list, labels=train_lab[0,:,:], transform=train_transforms);
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_cores, pin_memory=pin_memory);

    # create a validation data loader
    val_ds = ImageDataset(image_files=valid_list, labels=valid_lab[0,:,:], transform=val_transforms);
    val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=num_cores, pin_memory=pin_memory)

    # Create DenseNet121, CrossEntropyLoss and Adam optimizer
#     model = monai.networks.nets.DenseNet121(spatial_dims=2, in_channels=3, out_channels=2, pretrained=pretrain, progress=False).to(device)
    model = monai.networks.nets.resnet10(spatial_dims=2, num_classes=2, pretrained=pretrain, progress=False).to(device)
#     model = monai.networks.nets.ViT(in_channels=3, img_size=im_size, patch_size=pat_size, classification=True, num_classes=2, spatial_dims=2).to(device)

    # Loss Function
    loss_fx = torch.nn.BCEWithLogitsLoss() # DenseNet

    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    
    wb.watch(model, log='all')

    scaler = torch.cuda.amp.GradScaler()
    best_valid_score = 99999; # Initialize Loss
    lastmodel = None
    for epoch in range(max_epochs):
        epoch_loss = 0
        epoch_accuracy = 0

        for batch_data in train_loader:
            inputs, label = batch_data[0].to(device), batch_data[1].to(device);

            # Evaluate Model
            output = model(inputs);
            loss = loss_fx(output, label.float())

            # Update Gradient
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                # Evaluate Model
                output = model(inputs); epoch_prob = torch.nn.functional.softmax(output,dim=1)
                loss = loss_fx(output, label.float())
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            # Accuracy
            acc = (epoch_prob.argmax(dim=1) == label.argmax(dim=1)).float().mean()
            epoch_accuracy += acc / len(train_loader)
            epoch_acc_values.append(acc)

            # Loss
            epoch_loss += loss / len(train_loader)
            epoch_loss_values.append(epoch_loss)
    #         print(f"train loss: {loss.item():.4f}")
            wb.log({'train_loss': loss, 'train_acc': acc})

        if epoch % val_interval == 0: # Validation Interval
            with eval_mode(model):
                epoch_val_accuracy = 0; epoch_val_loss = 0;
                for val_data in val_loader:
                    val_images, val_labels = val_data[0].to(device), val_data[1].to(device)

                    val_output = model(val_images); val_prob = torch.nn.functional.softmax(val_output,dim=1);
                    val_loss = loss_fx(val_output, val_labels.float())

                    val_acc = (val_prob.argmax(dim=1) == val_labels.argmax(dim=1)).float().mean()
                    epoch_val_accuracy += val_acc / len(val_loader)
                    epoch_val_loss += val_loss / len(val_loader)
                    wb.log({'val_loss': val_loss, 'val_acc': val_acc})
            print(
                f"Epoch : {epoch+1} - train_loss : {epoch_loss:.4f} - train_acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
            )

            # Save Best Model
            if best_valid_score > epoch_val_loss:
                print("model saved")
                save_model(epoch, datatype, run_id)
                best_valid_score = epoch_val_loss

        else:    
            print(
                f"Epoch : {epoch+1} - train_loss : {epoch_loss:.4f} - train_acc: {epoch_accuracy:.4f} \n"
            )
            
    ###############
    #  Test Model #
    ###############
    
    modelfile = f'{datatype}-{run_id}-fold{f}.pth'; # HARDCODED
    checkpoint = torch.load(modelfile)

    # create a validation data loader
    test_ds = ImageDataset(image_files=test_list, labels=test_lab[0,:,:], transform=val_transforms)
    test_loader = DataLoader(test_ds, batch_size=batch_size, num_workers=num_cores, pin_memory=pin_memory)
    model.load_state_dict(checkpoint["model_state_dict"])

    with eval_mode(model):
        y_pred = torch.tensor([], dtype=torch.float32, device=device)
        y_prob = torch.tensor([], dtype=torch.float32, device=device)
        y = torch.tensor([], dtype=torch.long, device=device)
        test_acc = 0
        for test_data in test_loader:
            test_images, test_labels = test_data[0].to(
                device), test_data[1].to(device)

            outputs = model(test_images); probs = torch.nn.functional.softmax(outputs,dim=1);
#             test_loss = loss_fx(outputs, test_labels.float())
            test_acc += (probs.argmax(dim=1) == test_labels.argmax(dim=1)).float().mean();
            y_prob = torch.cat([y_prob, probs], dim=0)
            y_pred = torch.cat([y_pred, probs.argmax(dim=1)], dim=0)
            y = torch.cat([y, test_labels.argmax(dim=1)], dim=0);
        test_acc = test_acc / len(test_loader)
        alltargs.append(y.cpu().numpy())
        allpreds.append(y_pred.cpu().numpy())
        allscores.append(y_prob.cpu().numpy())
        print(f"Fold {f+1} Test Accuracy: ", test_acc.float())
#         wb.log({'test_loss': test_loss, 'test_acc': test_acc})
        wb.log({'test_acc': test_acc})
    butt = [ x + 2 for x in butt ]

wb.finish()    

print(classification_report(
    alltargs,
    allpreds,
    target_names=[d.name for d in Diagnosis]))

cm = confusion_matrix(
    alltargs,
    allpreds,
    normalize='true')

disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=[d.name for d in Diagnosis])
    
print("Test Accuracy: ", np.mean(alltargs == np.sign(allscores)).float())

disp.plot(ax=plt.subplots(1, 1, facecolor='white')[1])
    # wb.run.log_code(root=os.path.join(os.getcwd(),"CNN_SMART.ipynb")); 

Beginning Fold  1
test subs:  [9048 9045]
valid subs:  [9044 9054]
train case split:  404 : 606
valid case split:  101 : 101
test case split:  202 : 0 



Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)


Epoch : 1 - train_loss : 0.6699 - train_acc: 0.5974 - val_loss : 0.6934 - val_acc: 0.5491

model saved
Epoch : 2 - train_loss : 0.6128 - train_acc: 0.7045 - val_loss : 0.7236 - val_acc: 0.5491

Epoch : 3 - train_loss : 0.5391 - train_acc: 0.7785 - val_loss : 0.7355 - val_acc: 0.5491

Epoch : 4 - train_loss : 0.4280 - train_acc: 0.8698 - val_loss : 0.7410 - val_acc: 0.5491

Epoch : 5 - train_loss : 0.3261 - train_acc: 0.8981 - val_loss : 0.7979 - val_acc: 0.5491

Epoch : 6 - train_loss : 0.2226 - train_acc: 0.9680 - val_loss : 0.9188 - val_acc: 0.5491

Epoch : 7 - train_loss : 0.1874 - train_acc: 0.9805 - val_loss : 0.9319 - val_acc: 0.5491

Epoch : 8 - train_loss : 0.1532 - train_acc: 0.9873 - val_loss : 1.1065 - val_acc: 0.5491

Epoch : 9 - train_loss : 0.1217 - train_acc: 0.9961 - val_loss : 1.1069 - val_acc: 0.5491

Epoch : 10 - train_loss : 0.1219 - train_acc: 0.9922 - val_loss : 0.9442 - val_acc: 0.5491

Epoch : 11 - train_loss : 0.0940 - train_acc: 0.9971 - val_loss : 1.1692 - va

test_acc,▁▇▁▁▆▁█
train_acc,██████▁█████████████████████████████████
train_loss,▄▂▁▁▁▁█▂▁▁▁▁▃▂▁▁▁▇▂▂▁▁▁▃▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
val_acc,▁▁▇█▁▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂▃▁▁▃▄▃▂▆▃▆▂▂▁▁▃▄▂▂█▃▃▇▂▂▂▄▄▃▂▂▃▅▂▂▃▂▆▂▂
test_acc,0.45893
train_acc,1.0
train_loss,0.0013
val_acc,0.0
val_loss,1.43533


ValueError: Number of classes, 202, does not match size of target_names, 2. Try specifying the labels parameter

# Occulusion Sensitivity

In [ ]:
# cam = monai.visualize.CAM(nn_module=model_3d, target_layers="class_layers.relu", fc_layers="class_layers.out")
cam = monai.visualize.GradCAMpp(
    nn_module=model, target_layers="class_layers.relu"
)
# cam = monai.visualize.GradCAMpp(nn_module=model_3d, target_layers="class_layers.relu")
print(
    "original feature shape",
    cam.feature_map_size([1, 3] + list(im_size), device),
)
print("upsampled feature shape", [1, 3] + list(im_size))

occ_sens = monai.visualize.OcclusionSensitivity(
    nn_module=model, mask_size=4, n_batch=1, stride=4
)

# For occlusion sensitivity, inference must be run many times. Hence, we can use a
# bounding box to limit it to a 2D plane of interest (z=the_slice) where each of
# the arguments are the min and max for each of the dimensions (in this case CHWD).

train_transforms.set_random_state(42)
n_examples = 3
subplot_shape = [3, n_examples]
fig, axes = plt.subplots(*subplot_shape, figsize=(25, 15), facecolor="white")
items = np.random.choice(len(train_ds), size=len(train_ds), replace=False)

example = 0
for item in items:

    data = train_ds[
        item
    ]  # this fetches training data with random augmentations
    image, label = data[0].to(device).unsqueeze(0), data[1][1]
    y_pred = model(image); prob = torch.nn.functional.softmax(y_pred,dim=1);
    pred_label = prob.argmax(dim=1);
    
    # Only display preMCI
    if np.not_equal(label,1):
        continue

    img = image.detach().cpu().numpy()

    name = "actual: "
    name += "Responder" if np.equal(label,1) else "Non-Responder"
    name += "\npred: "
    name += "Responder" if np.equal(pred_label.cpu().numpy(),1) else "Non-Responder"
    name += f"\nResponder: {prob[0,1]:.3}"
    name += f"\nNon-Responder: {prob[0,0]:.3}"

    # run CAM
    cam_result = cam(x=image, class_idx=None)
    cam_result = cam_result

    # run occlusion
    occ_result, _ = occ_sens(x=image)
    occ_result = occ_result[..., pred_label];

    if isinstance(img, torch.Tensor):
            img = img.cpu().detach()
    ax = axes[0, example]
    im_show = ax.imshow(np.squeeze(img[0][0].T), cmap="jet", origin='lower')
    ax.set_title(name, fontsize=25)
    ax.axis("off")
    
    if isinstance(cam_result, torch.Tensor):
        cam_result = cam_result.cpu().detach()
    ax = axes[1, example]
    im_show = ax.imshow(np.squeeze(img[0][0].T), cmap="gray", origin='lower')
    im_show = ax.imshow(np.squeeze(cam_result[0][0].T), cmap="jet", origin='lower', alpha=0.5)
    ax.set_title("Grad-CAM++", fontsize=25)
    ax.axis("off")
    
    if isinstance(occ_result, torch.Tensor):
            occ_result = occ_result.cpu().detach()
    ax = axes[2, example]
    im_show = ax.imshow(np.squeeze(img[0][0].T), cmap="gray", origin='lower')
    im_show = ax.imshow(np.squeeze(occ_result[0][0].T), cmap="jet", origin='lower', alpha=0.5)
    ax.set_title("Occ. Sens", fontsize=25)
    ax.axis("off")
    
    example += 1
    if example == n_examples:
        break